In [114]:
import jax
import sys  
import jax.random as random
import jax.numpy as jnp
from jax.numpy import index_exp as index

sys.path.insert(0, '../')
from dibs.target import make_linear_gaussian_model, make_nonlinear_gaussian_model
from dibs.graph_utils import graph_to_mat #for graph sampling
from dibs.models import LinearGaussian #for parameter sampling

key = random.PRNGKey(123)
print(f"JAX backend: {jax.default_backend()}")

JAX backend: cpu


In [139]:
subkey

DeviceArray([ 961309823, 1704866707], dtype=uint32)

In [152]:
#try to see how to do the for loop and in which type to store the subkeys for sampling multiple parameters
seed = 1701
num_steps = 100
key = jax.random.PRNGKey(seed)
print(key)
#l = jnp.arange(num_steps)
l = []
print(l)
print(type(l))
for i in range(num_steps):
    key, subkey = jax.random.split(key)
    print(i)
    print(key)
    l.append(subkey)
    #l = l.at[i].set(subkey)
    print(subkey)
l 

[   0 1701]
[]
<class 'list'>
0
[  56197195 1801093307]
[ 961309823 1704866707]
1
[1948314926   18442562]
[3607299241 1501002339]
2
[3012589754 2847679230]
[2704933330 1409745451]
3
[3389310233  100302506]
[2141046588 4135594142]
4
[1209106627 2008329078]
[2452947530 2685855115]
5
[665794805 409583056]
[3528035545 4263362951]
6
[ 496085882 3433032244]
[1993612224 1385908671]
7
[1120391938 4083081944]
[3609638420  835929708]
8
[1964724363 1570491772]
[2816395849 2169436777]
9
[  73993845 3462729352]
[2470002977 2956761710]
10
[1597116053 2870019329]
[1184564883  492457053]
11
[ 595063410 4046883715]
[1055853113 2001004309]
12
[2058455463 2473782384]
[3984770594 2892741285]
13
[ 518365813 2738773632]
[2253285099 1607622566]
14
[2773509240 2037829253]
[4271157486 3762127267]
15
[3548303316 3589081137]
[3417485873 3613373643]
16
[ 144777565 3436797870]
[2071496299 2398221226]
17
[3315133881 1734038502]
[3503849617 2949005368]
18
[1486159605 2849446752]
[1041153962 2777832569]
19
[ 65894703

[DeviceArray([ 961309823, 1704866707], dtype=uint32),
 DeviceArray([3607299241, 1501002339], dtype=uint32),
 DeviceArray([2704933330, 1409745451], dtype=uint32),
 DeviceArray([2141046588, 4135594142], dtype=uint32),
 DeviceArray([2452947530, 2685855115], dtype=uint32),
 DeviceArray([3528035545, 4263362951], dtype=uint32),
 DeviceArray([1993612224, 1385908671], dtype=uint32),
 DeviceArray([3609638420,  835929708], dtype=uint32),
 DeviceArray([2816395849, 2169436777], dtype=uint32),
 DeviceArray([2470002977, 2956761710], dtype=uint32),
 DeviceArray([1184564883,  492457053], dtype=uint32),
 DeviceArray([1055853113, 2001004309], dtype=uint32),
 DeviceArray([3984770594, 2892741285], dtype=uint32),
 DeviceArray([2253285099, 1607622566], dtype=uint32),
 DeviceArray([4271157486, 3762127267], dtype=uint32),
 DeviceArray([3417485873, 3613373643], dtype=uint32),
 DeviceArray([2071496299, 2398221226], dtype=uint32),
 DeviceArray([3503849617, 2949005368], dtype=uint32),
 DeviceArray([1041153962, 27

100

In [116]:
key, subk = random.split(key)

In [3]:
key, subk = random.split(key)
data, model = make_nonlinear_gaussian_model(key=subk, n_vars=20, graph_prior_str="sf")


### Step one is to sample a random Graph:

In [16]:
data.x #should have shape of 100*20 

100

In [18]:
from dibs.models.graph import ErdosReniDAGDistribution


In [30]:
g_gt = ErdosReniDAGDistribution(n_vars = 20).sample_G(key = subk, return_mat=False)

In [31]:
g_gt

In [65]:
g_gt.topological_sorting()

[6, 11, 12, 15, 17, 4, 10, 19, 8, 0, 5, 13, 7, 14, 9, 2, 1, 16, 3, 18]

In [80]:
len(g_gt.vs)

20

In [101]:
parent_edges = g_gt.incident(19, mode='in')
print(parent_edges)
parents = list(g_gt.es[e].source for e in parent_edges)

[9, 27]


In [102]:
parents

[6, 15]

In [36]:
g_gt_mat = jnp.array(graph_to_mat(g_gt)) #why do we need this while we have a adj matrix already from the sample_G function?

In [39]:
g_gt_mat # 20*20 shape since we have 20 nodes on the graph

DeviceArray([[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
             [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
             [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
             [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
             [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0],
             [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
             [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
             [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
             [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
             [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
             [1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0],
             [0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
             [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
             [0, 0, 0, 0,

### Step two is to sample the parameters: 

In [157]:
theta = LinearGaussian(graph_dist = ErdosReniDAGDistribution(
            n_vars=20, 
            n_edges_per_node=2)).sample_parameters(key=subk, n_vars = 20)

In [236]:
#try to see how to do the for loop and in which type to store the subkeys for sampling multiple parameters
# I would add the number of timepoints and do a trick to avoid the same seeds giving rise to the same parameters. 
seed = 1
num_obs = 2
key = jax.random.PRNGKey(seed)
print(key)
#l = jnp.arange(num_steps)
#l = []
#print(l)
#print(type(l))
observations = jnp.array(object= jnp.empty(shape=(0,20)))
for i in range(num_steps):
    key, subk = jax.random.split(key)
    print(i)
    print(key)
    theta = LinearGaussian(graph_dist = ErdosReniDAGDistribution(
           n_vars=20, 
           n_edges_per_node=2)).sample_parameters(key=subk, n_vars = 20)
    observation = LinearGaussian(graph_dist = ErdosReniDAGDistribution(
            n_vars=20, 
            n_edges_per_node=2)).sample_obs(key=subk, n_samples = 1, g = g_gt, theta = theta)
    observations = jnp.append(observations, values= observation, axis=0)
    print(observation)
    #print(theta)
    #print(key)
    #l.append(subkey)
    #l = l.at[i].set(subkey)
    #print(subkey) 

[0 1]
0
[2441914641 1384938218]
[[ 0.12619044 -0.3771744  -0.2625161   0.32729203  0.615659   -0.38788825
  -0.00281111  0.6932224  -0.5751246   0.4259092  -0.24562313 -0.26173314
   0.10807621 -0.9281473  -0.30882785  0.10668769 -0.49456137 -0.12753274
   1.096207   -0.26280993]]
1
[2698884502 3718367942]
[[-0.26192698 -1.183513   -0.29308     2.3202152  -0.16016826  0.10377014
  -0.07096305 -0.4253351   0.47655684  1.0644951  -0.32124123  0.14289615
  -0.02786717 -0.39078194  0.35374543  0.01186005 -1.1836246  -0.21596202
  -0.51576793  0.10380407]]


In [237]:
observations

DeviceArray([[ 0.12619044, -0.3771744 , -0.2625161 ,  0.32729203,
               0.615659  , -0.38788825, -0.00281111,  0.6932224 ,
              -0.5751246 ,  0.4259092 , -0.24562313, -0.26173314,
               0.10807621, -0.9281473 , -0.30882785,  0.10668769,
              -0.49456137, -0.12753274,  1.096207  , -0.26280993],
             [-0.26192698, -1.183513  , -0.29308   ,  2.3202152 ,
              -0.16016826,  0.10377014, -0.07096305, -0.4253351 ,
               0.47655684,  1.0644951 , -0.32124123,  0.14289615,
              -0.02786717, -0.39078194,  0.35374543,  0.01186005,
              -1.1836246 , -0.21596202, -0.51576793,  0.10380407]],            dtype=float32)

In [194]:
theta.shape

(20, 20)

In [164]:
theta

AttributeError: 'DeviceArray' object has no attribute 'type'

In [56]:
theta[0].shape

#I understand why we have 20 different thetas (corresponding to each Xi i = 1,...,20) but why each is of dimension 20? Just to make the matrix mult possible with the 
#adj matrix? In the end we end up unsing only the ones that have connections to the parents. 

(20,)

### Step three: Sample observations

In [224]:
observations = jnp.array(object= jnp.empty(shape=(0,20)))

In [225]:
observations

DeviceArray([], dtype=float32)

In [226]:
observation = LinearGaussian(graph_dist = ErdosReniDAGDistribution(
            n_vars=20, 
            n_edges_per_node=2)).sample_obs(key=subk, n_samples = 1, g = g_gt, theta = theta )

In [238]:
#checking the observations with sampling in the old way- the dimensions match 
obs_old = LinearGaussian(graph_dist = ErdosReniDAGDistribution(
            n_vars=20, 
            n_edges_per_node=2)).sample_obs(key=subk, n_samples = 2, g = g_gt, theta = theta )

(20,)

In [228]:
observation.shape

(1, 20)

In [233]:
observations = jnp.append(observations, values= observation, axis=0)

In [235]:
observations.shape

(2, 20)

In [61]:
# we have 100 observations that each has dimension 20 
observations[9].shape

(20,)

In [94]:
 x = jnp.zeros((100, len(g_gt.vs)))

In [96]:
x.shape

(100, 20)

In [104]:
mean = x[:, jnp.array(parents)] @ theta[jnp.array(parents), 19] 

In [107]:
theta[jnp.array(parents), 19]

DeviceArray([ 3.2422197, -0.8611006], dtype=float32)

In [106]:
len(mean)

100

In [99]:
x.at[index[:, 0]].set(5)

DeviceArray([[5., 0., 0., ..., 0., 0., 0.],
             [5., 0., 0., ..., 0., 0., 0.],
             [5., 0., 0., ..., 0., 0., 0.],
             ...,
             [5., 0., 0., ..., 0., 0., 0.],
             [5., 0., 0., ..., 0., 0., 0.],
             [5., 0., 0., ..., 0., 0., 0.]], dtype=float32)